In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import shutil as st
import zipfile as zp

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#Making directories for future usage and seperating data for both cas and dogs

os.chdir('/kaggle/working/')

os.mkdir('cats')
os.mkdir('dogs')
os.mkdir('data_set')

In [ ]:
#Extracting training data to desired directory

with zp.ZipFile('../input/dogs-vs-cats/train.zip', 'r') as zip_ref:
    zip_ref.extractall('./data_set')

In [ ]:
#transfering cats and dogs to specified directory respectively

os.chdir('/kaggle/working/data_set/train/')
data_names = os.listdir()
for images in data_names:
    if images[:3] == 'cat':
        st.move(images,'/kaggle/working/cats/')
    elif images[:3] == 'dog':
        st.move(images,'/kaggle/working/dogs/')


In [ ]:
#Building model architecture

from keras import layers
from keras import models
from keras.models import Sequential

model = Sequential()
model.add(layers.Conv2D(32,(3,3), activation = 'relu', input_shape = (150,150,3)))
model.add(layers.MaxPooling2D(2,2))
model.add(layers.Conv2D(64,(3,3), activation = 'relu'))
model.add(layers.MaxPooling2D(2,2))
model.add(layers.Conv2D(128,(3,3), activation = 'relu'))
model.add(layers.MaxPooling2D(2,2))
model.add(layers.Conv2D(128,(3,3), activation = 'relu'))
model.add(layers.MaxPooling2D(2,2))
model.add(layers.Flatten())
model.add(layers.Dense(512, activation = 'relu'))
model.add(layers.Dense(1, activation = 'sigmoid'))

model.summary()

In [ ]:
#Compiling model

from keras import optimizers

model.compile(loss = 'binary_crossentropy', optimizer = optimizers.RMSprop(lr=1e-4), metrics = ['acc'])

In [ ]:
#Correcting my working directory before preprocessing

os.chdir('/kaggle/working/data_set')
os.rmdir('train')
os.chdir('/kaggle/working/')
os.rmdir('data_set')
os.chdir('/kaggle/working/')

In [1]:
# Generating images in desired size

from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255)
test_datagen = ImageDataGenerator(rescale = 1./255)

train_generator = train_datagen.flow_from_directory('./', target_size = (150,150), batch_size = 20, class_mode = 'binary')

KeyboardInterrupt: 

In [ ]:
# Running model on train dataset

functional_model = model.fit_generator(train_generator, steps_per_epoch = 100, epochs = 30)

In [ ]:
# Opening test data

import csv

os.chdir('/kaggle/working/')
os.mkdir('test1')
with zp.ZipFile('../input/dogs-vs-cats/test1.zip', 'r') as zip_ref:
    zip_ref.extractall('/kaggle/working/test1')

In [ ]:
# Performing model on test dataset

from keras.preprocessing import image
from matplotlib.pyplot import imshow

os.chdir('/kaggle/working/test1/test1/')

with open('result.csv', 'w', newline = '') as file:
    fieldnames = ['id','label']
    writer = csv.DictWriter(file, fieldnames = fieldnames)
    writer.writeheader()
    for i in range(1,12501):
        impath = ('/kaggle/working/test1/test1/{}.jpg'.format(i))
        img = image.load_img(impath, target_size = (150,150))
        x = image.img_to_array(img)
        x /= 255
        x = np.expand_dims(x, axis = 0)
        writer.writerow({'id':i, "label":model.predict_classes(x)[0][0]})

                     
                     


In [ ]:
# Saving results in the desired directory

st.move('/kaggle/working/test1/test1/result.csv','/kaggle/working/result.csv')

